In [1]:
# hyper_parameter tunning

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
from tensorflow import keras

In [5]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [6]:
from sklearn.datasets import fetch_california_housing

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
housing = fetch_california_housing()

In [10]:
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

In [12]:
class_names = ["T-shit/top", "Trouser", "Pullover", "Dress", "Coat",
              "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [13]:
scaler = StandardScaler()

In [14]:
X_train = scaler.fit_transform(X_train)

In [15]:
X_valid = scaler.transform(X_valid)

In [16]:
X_test = scaler.transform(X_test)

In [17]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [18]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='mse', optimizer=optimizer)
    return model

In [19]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [20]:
%%time
keras_reg.fit(X_train, y_train, epochs=100,
             validation_data=(X_valid, y_valid),
             callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 46us/sample - loss: 1.1399 - val_loss: 24.4309
Epoch 2/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.7393 - val_loss: 3.2896
Epoch 3/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.5374 - val_loss: 0.6080
Epoch 4/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4796 - val_loss: 0.4532
Epoch 5/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4536 - val_loss: 0.4221
Epoch 6/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4373 - val_loss: 0.4144
Epoch 7/100
11610/11610 [==============================] - 0s 25us/sample - loss: 0.4256 - val_loss: 0.4025
Epoch 8/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.4178 - val_loss: 0.3938
Epoch 9/100
11610/11610 [==============================] - 0s 24us/sample - loss: 0.41

In [21]:
mse_test = keras_reg.score(X_test, y_test)

5160/5160 [==============================] - 0s 13us/sample - loss: 0.3473


In [22]:
X_new = X_test[:3]

In [23]:
y_pred = keras_reg.predict(X_new)

In [24]:
y_pred

array([0.6616515, 1.6505842, 4.104437 ], dtype=float32)

In [25]:
from scipy.stats import reciprocal

In [26]:
from sklearn.model_selection import RandomizedSearchCV

In [27]:
# param_distribs = {
#     "n_hidden": [0, 1, 2, 3],
#     "n_neurons": np.arange(1, 100),
#     "learning_rate": reciprocal(3e-4, 3e-2)
# } 이게 원문인데 속도를 빨리하기위해서 설정

In [28]:
param_distribs = {
    "n_hidden": [1, 2, 3],
    "n_neurons": np.arange(40, 45),
    "learning_rate": reciprocal(3e-4, 3e-2)
}

In [29]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)

In [30]:
# %%time
# rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
#                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [31]:
import sklearn
sklearn.__version__

'0.21.2'

In [32]:
%%time
rnd_search_cv.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid),
                 callbacks=[keras.callbacks.EarlyStopping(patience=2)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


7740/7740 [==============================] - 0s 58us/sample - loss: 1.7605 - val_loss: 0.8180
Epoch 2/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7492 - val_loss: 0.6902
Epoch 3/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6682 - val_loss: 0.6794
Epoch 4/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6227 - val_loss: 0.6877
Epoch 5/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5881 - val_loss: 0.5605
Epoch 6/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5563 - val_loss: 0.5246
Epoch 7/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5309 - val_loss: 0.5171
Epoch 8/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5110 - val_loss: 0.5072
Epoch 9/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.4933 - val_loss: 0.4714
Epoch 10/10
3870/3870 [==============================] - 0s 13us/s

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


7740/7740 [==============================] - 0s 56us/sample - loss: 1.5698 - val_loss: 0.8937
Epoch 2/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.7232 - val_loss: 0.6627
Epoch 3/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6623 - val_loss: 0.7438
Epoch 4/10
3870/3870 [==============================] - 0s 13us/sample - loss: 0.6249
[CV]  learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42, total=   1.2s
[CV] learning_rate=0.001683454924600351, n_hidden=1, n_neurons=42 ....
Train on 7740 samples, validate on 3870 samples
Epoch 1/10
7740/7740 [==============================] - 0s 57us/sample - loss: 2.0150 - val_loss: 7.4186
Epoch 2/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7936 - val_loss: 3.2821
Epoch 3/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6761 - val_loss: 1.3613
Epoch 4/10
7740/7740 [==============================] - 0s 27us/sample - loss: 0.6329 - val_lo

...

7740/7740 [==============================] - 0s 28us/sample - loss: 0.8471 - val_loss: 0.7278
Epoch 4/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.7128 - val_loss: 0.6497
Epoch 5/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6644 - val_loss: 0.6537
Epoch 6/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6384 - val_loss: 0.6023
Epoch 7/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.6171 - val_loss: 0.5971
Epoch 8/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5997 - val_loss: 0.5932
Epoch 9/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5844 - val_loss: 0.5976
Epoch 10/10
3870/3870 [==============================] - 0s 13us/sample - loss: 0.5649
[CV]  learning_rate=0.0006930605663535878, n_hidden=1, n_neurons=40, total=   2.5s
[CV] learning_rate=0.0012178834831452913, n_hidden=2, n_neurons=44 ...
Train on 7740 samples, validate on 

7740/7740 [==============================] - 0s 28us/sample - loss: 0.6325 - val_loss: 7.9812
Epoch 3/10
7740/7740 [==============================] - 0s 28us/sample - loss: 0.5663 - val_loss: 1.0344
Epoch 4/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4517 - val_loss: 0.5222
Epoch 5/10
7740/7740 [==============================] - 0s 29us/sample - loss: 0.4178 - val_loss: 0.4184
Epoch 6/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.4027 - val_loss: 0.3701
Epoch 7/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3921 - val_loss: 0.3628
Epoch 8/10
7740/7740 [==============================] - 0s 30us/sample - loss: 0.3841 - val_loss: 0.3652
Epoch 9/10
3870/3870 [==============================] - 0s 12us/sample - loss: 0.3897
[CV]  learning_rate=0.005021425736625637, n_hidden=2, n_neurons=43, total=   2.4s
[CV] learning_rate=0.005021425736625637, n_hidden=2, n_neurons=43 ....
Train on 7740 samples, validate on 38

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.0min finished


11610/11610 [==============================] - 1s 61us/sample - loss: 0.6185 - val_loss: 1.9088
Epoch 2/10
11610/11610 [==============================] - 0s 27us/sample - loss: 0.4102 - val_loss: 0.4612
Epoch 3/10
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3702 - val_loss: 0.3281
Epoch 4/10
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3536 - val_loss: 0.3676
Epoch 5/10
11610/11610 [==============================] - 0s 28us/sample - loss: 0.3467 - val_loss: 0.5500
Wall time: 1min 2s


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x000002153EC31588>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000002154080CCC0>,
                                        'n_hidden': [1, 2, 3],
                                        'n_neurons': array([40, 41, 42, 43, 44])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [33]:
rnd_search_cv.best_params_

{'learning_rate': 0.026584732357599207, 'n_hidden': 3, 'n_neurons': 42}

In [34]:
rnd_search_cv.score(X_test,y_test)

5160/5160 [==============================] - 0s 15us/sample - loss: 0.3432


-0.34319735215615854

In [35]:
rnd_search_cv.best_estimator_

In [36]:
model=rnd_search_cv